In [2]:
import pandas as pd
from poem_env import PoemGenerationEnv
from poem_dql import DQLAgent
import numpy as np

2024-12-29 08:50:05.871064: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


done


In [13]:
df = pd.read_csv('../data/temp_romanian_words.csv')
df = df[df['joint_syllables'] != 'not-found']
df.to_csv('../data/temporary_romanian_words.csv', index=False)

In [14]:
# Clean data
df = pd.read_csv('../data/temporary_romanian_words.csv')
syls = list(df['syllables'])
new_syls = []
for s in syls:
    s = eval(s)
    i=0
    while i < (len(s)):
        if '(' in s[i]:
            break
        i+=1
    new_syls.append(s[:i])
            
print (len(new_syls), len(df))
df['syllables'] = new_syls
df.to_csv('../data/temporary2_romanian_words.csv', index=False)

12719 12719


In [3]:
df = pd.read_csv('../data/temporary2_romanian_words.csv')

In [3]:
# Initialize the environment
env = PoemGenerationEnv(df, num_verses=2, syllables_per_verse=6, rhyme_data=3)

# Interact with the environment
state = env.reset()
done = False

while not done:
    action = env.sample_action()  # Random action (replace with DQL model's action)
    state, reward, done = env.step(action)
    print(f"State: {state}")
    print(f"Reward: {reward}")
    print(f"Done: {done}")

State: {'poem': ['bibliomanie'], 'verse': [], 'syllables_left': 6, 'verse_index': 1}
Reward: 0
Done: False
State: {'poem': ['bibliomanie'], 'verse': ['camarilă'], 'syllables_left': 2, 'verse_index': 1}
Reward: 1
Done: False
State: {'poem': ['bibliomanie', 'camarilă clujean'], 'verse': [], 'syllables_left': 6, 'verse_index': 2}
Reward: 50
Done: True


In [5]:
# Hyperparameters
episodes = 500
batch_size = 32
state_size = 3  # For example: poem length, syllables left, verse index
action_size = len(env.vocab)  # Vocabulary size
target_update_freq = 10

In [6]:

# Initialize the environment and agent
env = PoemGenerationEnv(df, num_verses=4, syllables_per_verse=6, rhyme_data=3)
agent = DQLAgent(state_size, action_size)

for episode in range(episodes):
    # Reset environment at the start of each episode
    state = env.reset()
    state_vector = np.array([state["verse_index"], state["syllables_left"], len(state["poem"])])
    total_reward = 0

    done = False
    while not done:
        # Choose an action
        action = agent.act(state_vector)

        # Take the action in the environment
        next_state, reward, done = env.step(action)
        next_state_vector = np.array([next_state["verse_index"], next_state["syllables_left"], len(next_state["poem"])])

        # Store experience
        agent.remember(state_vector, action, reward, next_state_vector, done)

        # Update state
        state_vector = next_state_vector
        total_reward += reward

        # Replay and train
        agent.replay(batch_size)

    # Update target network periodically
    if episode % target_update_freq == 0:
        agent.update_target_network()
        # Save the Q-network weights
        agent.q_network.save_weights("q_network.weights.h5")    

    print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")


Episode 1/500, Total Reward: -15, Epsilon: 1.00
Episode 2/500, Total Reward: -15, Epsilon: 1.00
Episode 3/500, Total Reward: -15, Epsilon: 1.00
Episode 4/500, Total Reward: -15, Epsilon: 1.00
Episode 5/500, Total Reward: -15, Epsilon: 1.00
Episode 6/500, Total Reward: -15, Epsilon: 1.00
Episode 7/500, Total Reward: -15, Epsilon: 1.00
Episode 8/500, Total Reward: -15, Epsilon: 1.00
Episode 9/500, Total Reward: -15, Epsilon: 1.00
Episode 10/500, Total Reward: -15, Epsilon: 1.00
Episode 11/500, Total Reward: -15, Epsilon: 1.00
Episode 12/500, Total Reward: -15, Epsilon: 1.00
Episode 13/500, Total Reward: -15, Epsilon: 0.98
Episode 14/500, Total Reward: -15, Epsilon: 0.97
Episode 15/500, Total Reward: -15, Epsilon: 0.96
Episode 16/500, Total Reward: -15, Epsilon: 0.95
Episode 17/500, Total Reward: -15, Epsilon: 0.94
Episode 18/500, Total Reward: -15, Epsilon: 0.92
Episode 19/500, Total Reward: -15, Epsilon: 0.89
Episode 20/500, Total Reward: -15, Epsilon: 0.87
Episode 21/500, Total Reward:

: 

In [6]:
env = PoemGenerationEnv(df, num_verses=4, syllables_per_verse=6, rhyme_data=3)
agent = DQLAgent(state_size, action_size)
agent.q_network.load_weights("q_network.weights.h5")

state = env.reset()
state_vector = np.array([state["verse_index"], state["syllables_left"], len(state["poem"])])

done = False
generated_poem = []

while not done:
    action = agent.act(state_vector)
    next_state, reward, done = env.step(action)
    state_vector = np.array([next_state["verse_index"], next_state["syllables_left"], len(next_state["poem"])])
    generated_poem = next_state["poem"]

print("Generated Poem:")
print("\n".join(generated_poem))


Generated Poem:



/opt/miniconda3/envs/poem-mode/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
